# MNIST VAE

The most common and well understood problem in machine learning is that of the hand written digits dataset called MNIST. It contains 60,000 examples of hand written digits and let's will use it as a guide to introduce you to the Flux library, that will cover a basic overview as well as what makes it hackable.

This walkthrough is going to be a little different from the basic MNIST classifiers in that we will focus more of our efforts on writing a Varitational Autoencoder (VAE). It uses very basic building blocks to make a classifier as before, with the difference that we will use it to enocode data about what makes the digits, the digits. Then we will explore how to generatively create these numbers using our learned encodings and latent vectors that we sample from a distribution. 

<b>TL;DR we will make a model which will train over bunch of numbers, and then have it generate a number, as best it can.</b>

Variational Autoencoders exploit the fact that rather than discriminators, we can think of neural networks as a data structure which is capable of holding the information we give it, in some encoded form. Sampling from a distribution would give us the latent variables that we send to the decoder to generate the images. GANs use this idea to create generative as well as discriminative models which can perform sophisticated tasks like telling which picture of a cat is that of a real cat rather than one made by a computer somewhere.

![title](imgs/encoder.jpeg)

Let's start with loading the dataset to get an idea of what we're dealing with here.


In [16]:
using Flux, Flux.Data.MNIST

The MLDatasets package in julia provides a neat and easy way of working with a lot of popular, freely available datasets. It's implementation in Flux allows us to provide a clean interface to getting our data easily just by calling the `MNIST.images()` function and we are good to start working with MNIST.

In [17]:
imgs = MNIST.images();

In [34]:
two = imgs[6]

In [35]:
size(two)

(28, 28)

Notice the dimensions of the images, we'll need those when we define our model (more on that in a minute). Also notice, the images are all grayscale with intensity values between 0 (black, no intensity) and 255 (completely white). This is interesting as it already tells us how our data is going to be bound. 60,000 images of size 28 X 28 where every pixel value is bound between 0 and 255. Neat! We're already a good way into understanding our data. 

# Handling Data with care

60,000 is a lot of images to deal with in one go. This will also allow us to introduce stochasticity into our training, which can help us get around saddle points. Here, X takes all the images and converts the pixels with value greater than 0.5 to true and the rest to false. Every column of X repsents an image. 

Further, N is the number of images we want to partition and M is the size of this minibatch. There are a lot of theories behind how to choose the appropriate mini batch for learning. Typically, batches of between `2` and `32` are chosen.

We break down the batch using the `partition` function in `Base.Iterators`. It behaves similar to a step in a loop for our use here.

In [43]:
# Load data, binarise it, and partition into mini-batches of M.
X = float.(hcat(vec.(imgs)...)) .> 0.5
N, M = size(X, 2), 100
data = [X[:,i] for i in Iterators.partition(1:N,M)];

In [53]:
data |> size, data[1] |> size

((600,), (784, 100))

Success! The images are now arranged in 600 batches of 100.

# Onto the Mathematics

![title](imgs/oprah_ml.gif)

In [1]:
using Flux: throttle, params
using Juno: @progress # import statements for functions we will use throughout

Before we get ahead of ourselves, it is important to remind ourselves that what we are dealing with are basically just distributions in a large vector space that we are explicitly giving meaning to. And therefore, all laws that govern these distributions will have to be respected. This also means we can take advantage of them, especially if we represent them in constructs that have certain desirable qualities. Here, we have binarised our data beforehand and our data has meaning explicitly in the position it acquires within the 28 X 28 grid of its image. 

We will have to calculate the likelihood of a particular pixel being activated and for that we will make use of the `logpdf` function which is part of the `Distributions` package. The `Bernoulli` distribution is what we will use to model the likelihood function so we have a nice evenly distributed function to sample from when we want to create our latent vectors. 

It is really imoprtant to point out that we want to ideally have our network train encodings in a roughly gaussian distribution so we can later just sample our latent vectors from there rather than having to ues the images. This is what allows our autoencoder to be generative.

In [14]:
using Distributions
import Distributions: logpdf
logpdf(b::Bernoulli, y::Bool) = y * log(b.p + eps()) + (1 - y) * log(1 - b.p + eps())

logpdf (generic function with 62 methods)

Julia provides powerful tools to extend the functionality of existing functions using it's multiple dispatch system which will pick the the version of a function best suited to perform the task with defined behaviour that can take advantage of the characteristics of the input we give it.

As you've noticed, we've "overloaded" the `logpdf` function here. This is because, we know our binarised data is going to show disparity and will have its `p` value close to either `0` or `1`, and we want to make it stable around those regions. We do this by adding a perturbation `ϵ` and calculating the log likelihood of the pixel being 0 or 1.

# _The Model_

![title](imgs/ml_models.jpg)

We need to decide the metric we will use to decide how our network is performing. This is usually done by something formally known as the loss function or the cost function. In our case, we have to measure the loss of how closely the network recreated the image we input and our second constraint, about having the distribution remaning fairly gaussian. 

In order to have a good chance of remaining within our second constraint and modeling a posterior, we will go towards variational inference and its many forms. 

The KL version of the Monte Carlo estimator is a good starting point to keep our approximation of the posterior in check as it allows for fast convergence early on in the optimisation process. Furthermore, we can calculate the entopy in its closed form, since our approximate of the conditional probability is tractable. This is a common reason to prefer KL or the entropy form of the Monte Carlo estimator, even though a full Monte Carlo approach can yield a higher variance. It can make the optimisation more efficient. Remember, we are only interested in the gradient of the the lower bound and not the values themselves.

![title](http://andymiller.github.io/images/posts/mc_gradient_variance/gradient_variance.gif)

And if all that were not enough, this is provided us through julia's excellent automatic differentiation capabilities. 

The ability to efficiently compute gradients is the secret sauce that ties this all together. The cherry on the top is that the code looks just like normal julia, and represents the mathematics very closely, making it expressable for us, the end users.

Phew, that is altogether too long between code. Just to get back on track, I think its a nice idea to define our encoder now. At this point, all we need to worry about is that it takes our image of the digit as an input. 

In [60]:
# Latent dimensionality, # hidden units.
Dz, Dh = 5, 500

(5, 500)

In [102]:
# Components of recognition model / "encoder" MLP.
A, μ, logσ = Dense(28^2, Dh, tanh), Dense(Dh, Dz), Dense(Dh, Dz)
g(X) = (h = A(X); (μ(h), logσ(h)))
z(μ, logσ) = μ + exp(logσ) * randn()

z (generic function with 1 method)

`A` here is our input layer. We are going to use an MLP for our encoder here. `tanh` is the activation function that we apply to `A`. `μ` and `logσ` here are the next layers. Notice how our network diverges here, this is done to add a bit of noise into our learned latent variables.

`g` takes the input and propagates it through our model. `z` is the transform we apply to the two layers μ and logσ

In [91]:
# Generative model / "decoder" MLP.
f = Chain(Dense(Dz, Dh, tanh), Dense(Dh, 28^2, σ))

Chain(Dense(5, 500, tanh), Dense(500, 784, NNlib.σ))

Now to put it all together, `f` is our decoder, which is another simple MLP. This uses Flux's `Chain` function, which allows us to propagate an input through a `Chain` of functions, with the output of one, being the input to the other. This is especially suited for deep learning, as creating deeper (i.e models with more and more layers) models allows to accomodate more parameters, enabling it to learn higher order features. That, however, is the topic a different blog post.

It is also worth mentioning that the `Chain` function takes general layers as inputs, which are simple julia functions themselves, meaning we can use Chain with arbitrary julia functions as long as the functions are compatible with each other.

![title](imgs/deeper_meme.jpg)

I would encourage you to try using different methods to explore both the models as well as the loss functions. I must warn you of the deep rabbit hole that you will inevitably stumble across.

Now we shall define our loss function. This directly corresponds to the discussion we had earlier.

In [105]:
# KL-divergence between approximation posterior and N(0, 1) prior.
kl_q_p(μ, logσ) = 0.5 * sum(exp.(2 .* logσ) + μ.^2 - 1 .+ logσ.^2)

# logp(x|z) - conditional probability of data given latents.
logp_x_z(x, z) = sum(logpdf.(Bernoulli.(f(z)), x))

# Monte Carlo estimator of mean ELBO using M samples.
L̄(X) = ((μ̂, logσ̂) = g(X); (logp_x_z(X, z.(μ̂, logσ̂)) - kl_q_p(μ̂, logσ̂)) / M)

loss(X) = -L̄(X) + 0.01 * sum(x->sum(x.^2), params(f))

loss (generic function with 1 method)

The `.`s you see here are the result of this concept known as broadcasting. Essentially, it allows element wise operations of a function on its inputs. You might be fmailiar to a similar concept of `map` if you're coming from python or java and the like. 

Also also, the `->` arrows you see sprinkled throughout are to make something called as anonymous functions. They are functions that are called at runtime. This again has an analog in the `lambda` functions from java or python.

# Training the VAE

Having a function that we can use to sample from the distribution and send it to the model will also be necessary. So let us define a `modelsample` function that does that for us.

In [106]:
# Sample from the learned model.
modelsample() = rand.(Bernoulli.(f(z.(zeros(Dz), zeros(Dz)))))

modelsample (generic function with 1 method)

Time to train the model. The `evalcb` function is a callback that is called every 30 seconds. It calculates the loss to give us an idea of what the model is doing. We will train it for 20 epochs, that is, the entire dataset will go through our VAE 20 times. We will give it a chance to see every image 20 times; notice the lack of any `Dropout`. 

We will use the well tested `ADAM` optimiser for this run.

In [107]:
evalcb = throttle(() -> @show(-L̄(X[:, rand(1:N, M)])), 30)
opt = ADAM(params(A, μ, logσ, f))
@progress for i = 1:20
  info("Epoch $i")
  Flux.train!(loss, zip(data), opt, cb=evalcb)
end

MethodError: MethodError: Cannot `convert` an object of type Array{Any,1} to an object of type Float64
Closest candidates are:
  convert(::Type{T<:Number}, !Matched::T<:Number) where T<:Number at number.jl:6
  convert(::Type{T<:Number}, !Matched::Number) where T<:Number at number.jl:7
  convert(::Type{T<:Number}, !Matched::Base.TwicePrecision) where T<:Number at twiceprecision.jl:250
  ...

That was a lot, let's see how we did.

# The Output

In [109]:
using Images

MethodError: MethodError: no method matching pipe_writer(::IJulia.IJuliaStdio{Base.PipeEndpoint})
The applicable method may be too new: running in world age 27557, while current world is 28835.
Closest candidates are:
  pipe_writer(::IJulia.IJuliaStdio) at /Users/dhairyagandhi/.julia/packages/IJulia/fDY4/src/stdio.jl:15 (method too new to be called from this world context.)
  pipe_writer(!Matched::Base.ProcessChain) at process.jl:344
  pipe_writer(!Matched::Base.Process) at process.jl:332
  ...

The Images package contains a number of tools and filter which are required for working with images. We will use the `Gray` function from here to convert our sample into the same data type as the one we get from MNIST, so we can pass it along to our model the same way.

In [111]:
img(x) = Gray.(reshape(x, 28, 28)) # converts an input to a grayscale image

img (generic function with 1 method)

In [112]:
# cd(@__DIR__)
sample_imgs = hcat(img.([modelsample() for i = 1:10])...)
save("sample.png", sample_imgs)

MethodError: MethodError: no method matching cat_nested(::Base.Broadcast.Broadcasted{Flux.Tracker.TrackedStyle,Nothing,typeof(tanh),Tuple{Base.Broadcast.Broadcasted{Flux.Tracker.TrackedStyle,Nothing,typeof(+),Tuple{TrackedArray{…,Array{Float64,1}},TrackedArray{…,Array{Float64,1}}}}}})
Closest candidates are:
  cat_nested(::Any, !Matched::Base.Broadcast.Broadcasted) at broadcast.jl:321
  cat_nested(::Any, !Matched::Tuple{}, !Matched::Tuple{}) at broadcast.jl:328
  cat_nested(::Any, !Matched::Tuple{}, !Matched::Any) at broadcast.jl:327
  ...

In [114]:
sample_imgs

UndefVarError: UndefVarError: sample_imgs not defined

# Summary

That wasn't too bad for a start. Of course, there are many different ways to approach any problem. The beauty of julia truly lies within its ease to extend its functionality without any cost of development. This means your work is not bottlenecked by the language or its implementation, giving you the space to experiment and develop without coming in the way.